In [112]:
import sys
import os
import json
!conda install --yes --prefix {sys.prefix} elasticsearch
!conda install --yes --prefix {sys.prefix} elasticsearch-dsl
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Search

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [113]:
def load_json_file(file):
    data = []
    if os.path.exists(file):
        for line in open(file, 'r'):
            data.append(json.loads(line))
        return data
    else:
        return("File does not exist")

In [114]:
def get_list_by_chunk_size(original_list, batch_size):
    # looping till length equals batch_size
    for i in range(0, len(original_list), batch_size):  
        yield original_list[i:i + batch_size] 

In [115]:
def bulk_insert_data_to_es(elasticsearch_connection, data, index, bulk_size=100):
    try:
        batch_data = get_list_by_chunk_size(data, bulk_size)
        for batch in batch_data:
            count = 0
            actions = []
            while count <= len(batch) - 1:
                action = {
                    "_index": index,
                    "_source": {}
                }
                action["_source"] = batch[count]
                actions.append(action)
                count = count + 1
            helpers.bulk(elasticsearch_connection, actions)
        return True
    except:
        e = sys.exc_info()
        print("Bulk insertion job failed")
        print(e)
        return False

In [116]:
def get_stopwords():
    stop_words = []
    with open(os.getcwd() + "/util" + "/sinhala_stopwords.txt", 'r') as f:
        for word in f:
            stop_words.append(word.strip())
    return stop_words

In [117]:
es_connection = Elasticsearch('http://localhost:9200')
es_connection.indices.delete(index="artists-*")

{'acknowledged': True}

In [118]:
stopwords = get_stopwords()
mapping = {
    "settings" : {
        "analysis" : {
            "analyzer" : {
                "sinhala_analyzer" : {
                    "type": "standard",
                    "stopwords": stopwords
                }
            }
        } 
    },
    "aliases": { 
        "artists": { 
            "is_write_index": True 
        }
    },
    "mappings" : {
        "properties" : {
            "awards" : {
                "properties" : {
                    "instituition" : {
                        "type" : "text",
                        "analyzer" : "sinhala_analyzer",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "ignore_above" : 256
                            }
                        }
                    },
                    "title" : {
                        "type" : "text",
                        "analyzer" : "sinhala_analyzer",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "ignore_above" : 256
                            }
                        }
                    }
                }
            },
            "bio" : {
                "type" : "text",
                "analyzer" : "sinhala_analyzer"
            },
            "birth" : {
                "type" : "text",
                "analyzer" : "sinhala_analyzer",
            },
            "death" : {
                "type" : "text",
                "analyzer" : "sinhala_analyzer",
            },
            "films" : {
                "properties" : {
                    "role" : {
                        "type" : "text",
                        "analyzer" : "sinhala_analyzer",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "ignore_above" : 256
                            }
                        }
                    },
                    "title" : {
                        "type" : "text",
                        "analyzer" : "sinhala_analyzer",
                        "fields" : {
                            "keyword" : {
                                "type" : "keyword",
                                "ignore_above" : 256
                            }
                        }
                    },
                    "year" : {
                      "type" : "long"
                    }
                }
            },
            "id" : {
                "type" : "long"
            },
            "name" : {
                "type" : "text",
                "analyzer" : "sinhala_analyzer",
                "fields" : {
                    "keyword" : {
                        "type" : "keyword",
                        "ignore_above" : 256
                    }
                }
            },
            "real_name" : {
                "type" : "text",
                "analyzer" : "sinhala_analyzer",
                "fields" : {
                    "keyword" : {
                      "type" : "keyword",
                      "ignore_above" : 256
                    }
                }
            }
      }
    }
}

In [119]:
es_connection.indices.create(index="artists-000001", body=mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'artists-000001'}

In [120]:
artist_data = load_json_file(os.getcwd() + '/corpus/artists.json')
bulk_insert_data_to_es(es_connection, artist_data, "artists")

True